In [1]:
import sqlite3
import pandas as pd
import os
import mido
from mido import MidiFile
from midi_player import MIDIPlayer
from midi_player.stylers import basic, cifka_advanced
# import musicbrainzngs
from magenta.models.music_vae import data

os.chdir('../')
os.chdir('../')

from plagdet.src.defaults import *
from plagdet.scripts.synthetic_data.monophonic_melody_extractor import MonophonicMelodyExtractor
from plagdet.scripts.synthetic_data.pair_generator import PairGenerator
from plagdet.scripts.synthetic_data.utils import calculate_total_bars, get_midi_info


c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
c:\Users\cunn2\miniconda3\envs\magenta\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find 

In [7]:
import mido
path = r'C:\Users\cunn2\OneDrive\DSML\Project\thesis-repo\plagdet\data\synthetic_dataset\monophonic_midis\melodies'
# Get a list of all MIDI files in the directory
midi_files = [f for f in os.listdir(path) if f.endswith('.mid') or f.endswith('.midi')]

print(len(midi_files))

file1 = os.path.join(path, midi_files[7])

MIDIPlayer(file1, 500, styler=cifka_advanced, title='First Monophonic Melody')

1170


In [3]:
file2 = os.path.join(path, midi_files[9])

MIDIPlayer(file2, 500, styler=cifka_advanced, title='Second Monophonic Melody')

In [4]:
gen = PairGenerator()

mid1, mid2, metadata = gen.generate_pair(file1, file2)

mid1.save('mid1.mid')
mid2.save('mid2.mid')

for key, val in metadata.items():
    print(key, val)

MIDIPlayer('mid1.mid', 500, styler=cifka_advanced, title='First Monophonic Melody')


[2024-09-08 17:17:43] [INFO ] Original bar duration: 2.0
[2024-09-08 17:17:43] [INFO ] Target bar duration: 2.0
[2024-09-08 17:17:43] [INFO ] Original file: BPM: 120.0, Time Signature: (4, 4), Ticks per beat: 220
[2024-09-08 17:17:43] [INFO ] Shifting entire MIDI pitch by 0 semitones
[2024-09-08 17:17:43] [INFO ] Pitch shifting 1 notes
[2024-09-08 17:17:43] [INFO ] Pitch shifts: [8]
[2024-09-08 17:17:43] [INFO ] Scaling 1 notes
[2024-09-08 17:17:43] [INFO ] Scaling factors: [1.39417375]
[2024-09-08 17:17:43] [INFO ] Deleting 1 notes
[2024-09-08 17:17:43] [INFO ] Delete indices: [0]


segment_starting_bar 85
segment_starting_time_secs 170.0
segment_insertion_starting_bar 11
segment_insertion_starting_time_secs 22.0
midi_modifier_config MidiModifierConfig(use_shift_entire_midi_pitch=True, entire_shift_semitone=0, use_shift_selected_notes_pitch=True, selected_notes_pitch_shifts=[((23, 24), 8)], use_change_note_durations=True, note_scale_factors=[((14, 16), 1.394173754999755)], use_delete_notes=True, notes_to_delete=[(2, 5)])


In [5]:
MIDIPlayer('mid2.mid', 500, styler=cifka_advanced, title='Second Monophonic Melody')


In [6]:
import shutil
from mido import MidiFile, MidiTrack, MetaMessage

# Duplicate dest.midi and halve its tempo
original_dest = 'dest.midi'
half_tempo_dest = 'half_tempo_dest.midi'
shutil.copy(original_dest, half_tempo_dest)

original_dest_one_track = 'dest_one_track.midi'
gen.condense_midi_to_one_track(original_dest, original_dest_one_track)

half_tempo_dest_one_track = 'half_tempo_dest_one_track.midi'
gen.condense_midi_to_one_track(half_tempo_dest, half_tempo_dest_one_track)

# Open the copied file and halve its tempo
midi = MidiFile(half_tempo_dest_one_track)
for track in midi.tracks:
    for msg in track:
        if msg.type == 'set_tempo':
            msg.tempo = msg.tempo*2

# Save the modified file
midi.save(half_tempo_dest_one_track)

# Run make_bars_same_duration on the original file
gen.scale_notes(original_dest_one_track, half_tempo_dest_one_track, 'adjusted_original.midi')

# Load and print information about all three files for comparison
def print_midi_info(file_path):
    midi = MidiFile(file_path)
    print(f"\nFile: {file_path}")
    print(f"Ticks per beat: {midi.ticks_per_beat}")
    for i, track in enumerate(midi.tracks):
        print(f"Track {i}:")
        for msg in track:
            if msg.type in ['set_tempo', 'time_signature']:
                print(f"  {msg}")

print_midi_info(original_dest)
print_midi_info(half_tempo_dest)
print_midi_info('adjusted_original.midi')

# Play all three files for auditory comparison
print(calculate_total_bars(original_dest))
print("\nPlaying original dest.midi:")
MIDIPlayer(original_dest_one_track, 500, styler=cifka_advanced, title='Original Dest')


FileNotFoundError: [Errno 2] No such file or directory: 'dest.midi'

In [ ]:

print("\nPlaying half tempo dest.midi:")
print(calculate_total_bars(half_tempo_dest_one_track))
MIDIPlayer(half_tempo_dest_one_track, 500, styler=cifka_advanced, title='Half Tempo Dest')



Playing half tempo dest.midi:
1.9375


In [ ]:
print("\nPlaying adjusted original.midi:")
MIDIPlayer('adjusted_original.midi', 500, styler=cifka_advanced, title='Adjusted Original')




Playing adjusted original.midi:


In [ ]:
# Load the MIDI file
MIDIPlayer(file, 500, styler=cifka_advanced)


In [ ]:
import logging
import numpy as np
import pretty_midi
import mido
import os
from mido import MidiFile, MidiTrack, MetaMessage

from typing import Optional, Union

from plagdet.scripts.synthetic_data.utils import calculate_total_bars, get_midi_info, set_midi_tempo, calculate_bar_duration
from plagdet.src.utils.log import configure_logging

configure_logging()
logger = logging.getLogger(__name__)

class SyntheticDataGenerator:

    def generate_pair(
        self,
        original_track_path: str,
        target_track_path: str,
    ):
        pass

    def condense_midi_to_one_track(self, input_path: str, output_path: str) -> None:
        # Load the MIDI file
        mid = MidiFile(input_path)
        
        # Create a new MIDI file and a single track
        new_mid = MidiFile(ticks_per_beat=mid.ticks_per_beat)
        new_track = MidiTrack()
        new_mid.tracks.append(new_track)
        
        # List to hold all messages with their absolute times
        all_messages = []
        
        # Iterate through all tracks and collect messages
        for track in mid.tracks:
            current_tick = 0
            for msg in track:
                current_tick += msg.time
                all_messages.append((current_tick, msg))
        
        # Sort messages by their absolute time
        all_messages.sort(key=lambda x: x[0])
        
        # Add messages to the new track, adjusting the time to be relative
        last_tick = 0
        for abs_tick, msg in all_messages:
            relative_time = abs_tick - last_tick
            new_msg = msg.copy(time=relative_time)
            new_track.append(new_msg)
            last_tick = abs_tick
        
        # Save the new MIDI file
        new_mid.save(output_path)
        
    def extract_bars(self, midi_path: str, dest_path: str, start_bar: int, num_bars: int = 2):

        total_bars_in_song = calculate_total_bars(midi_path)
        assert start_bar >= 1, f'Start bar must be at least 1. Got {start_bar}'
        assert start_bar + num_bars <= total_bars_in_song-1, f'Start bar + num bars must be less than total bars in song - 1. Got {start_bar} + {num_bars} = {start_bar + num_bars} > {total_bars_in_song-1}'
        # Load the MIDI file
        one_track_path = f'one_track.mid'
        self.condense_midi_to_one_track(midi_path, one_track_path)
        mid = MidiFile(one_track_path)
        
        # Calculate tempo and time signature
        tempo, time_signature, ticks_per_beat = get_midi_info(midi_path)
        bpm = mido.tempo2bpm(tempo)
        logger.info(f'Original file: BPM: {bpm}, Time Signature: {time_signature}, Ticks per beat: {ticks_per_beat}')

        # Calculate ticks per bar based on the time signature
        beats_per_bar = time_signature[0]
        ticks_per_bar = beats_per_bar * ticks_per_beat
        
        # Calculate start tick and end tick
        start_tick = (start_bar) * ticks_per_bar
        end_tick = start_tick + num_bars * ticks_per_bar
        
        # Create a new MIDI file
        new_mid = MidiFile(ticks_per_beat=ticks_per_beat)
        new_track = MidiTrack()
        new_mid.tracks.append(new_track)

        # Add tempo and time signature information
        tempo_in_microseconds = mido.bpm2tempo(bpm)
        new_track.append(MetaMessage('set_tempo', tempo=tempo_in_microseconds, time=0))
        new_track.append(MetaMessage('time_signature', numerator=time_signature[0], denominator=time_signature[1], clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))

        current_tick = 0    
        for msg in mid.tracks[0]:
            current_tick += msg.time
            if start_tick <= current_tick < end_tick:
                # Adjust message time if it's the first message
                if current_tick == start_tick:
                    msg.time = 0
                new_track.append(msg)
            elif current_tick >= end_tick:
                break

        # Save the new MIDI file
        new_mid.save(dest_path)

    # TODO: implement this, with the disturbance functions
    def apply_transformation(self, original_bar: np.ndarray, target_bar: np.ndarray):
        pass
    
    # TODO: this doesnt work at all

    def scale_notes(self, midi1_path: str, midi2_path: str, output_path: str):
        """
        Scales notes from midi1 so that the bar duration is the same as midi2, under the tempo and ticks of midi2.
        """
        # Get info for both MIDI files
        tempo1, time_sig1, tpb1 = get_midi_info(midi1_path)
        tempo2, time_sig2, tpb2 = get_midi_info(midi2_path)
        
        # Calculate bar durations
        bar_duration1 = calculate_bar_duration(tempo2, time_sig2, tpb1)
        bar_duration2 = calculate_bar_duration(tempo2, time_sig2, tpb2)
        
        # Calculate scaling factor
        scaling_factor = bar_duration2 / bar_duration1
        
        # Open MIDI files
        midi1 = MidiFile(midi1_path)
        midi2 = MidiFile(midi2_path)
        
        # Create new MIDI file with MIDI 2's properties
        new_midi = MidiFile(ticks_per_beat=tpb2)
        new_track = MidiTrack()
        new_midi.tracks.append(new_track)
        
        # Add tempo and time signature from MIDI 2
        new_track.append(MetaMessage('set_tempo', tempo=tempo2, time=0))
        new_track.append(MetaMessage('time_signature', numerator=time_sig2[0], denominator=time_sig2[1], clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))
        
        # Transfer and scale note-on messages from MIDI 1
        for track in midi1.tracks:
            for msg in track:
                if msg.type == 'note_on':
                    new_msg = msg.copy()
                    new_msg.time = int(msg.time * scaling_factor)
                    new_track.append(new_msg)
        
        # Save the new MIDI file
        new_midi.save(output_path)

    def insert_segment(self, segment_path: str, target_path: str, start_bar: int, dest_path: str) -> None:

        target_one_track_path = f'target_one_track.mid'
        segment_one_track_path = f'segment_one_track.mid'

        self.condense_midi_to_one_track(segment_path, segment_one_track_path)
        self.condense_midi_to_one_track(target_path, target_one_track_path)
        
        target_mid = MidiFile(target_one_track_path)

        tempo, time_signature, tpb_target = get_midi_info(target_one_track_path)
        
        # Scale the segment notes to match the target tempo and ticks per beat
        scaled_segment_path = 'temp_scaled_segment.mid'
        self.scale_notes(segment_one_track_path, target_one_track_path, scaled_segment_path)
        segment_mid = MidiFile(scaled_segment_path)
        
        num_bars_in_segment = calculate_total_bars(scaled_segment_path)
        ticks_per_bar = time_signature[0] * tpb_target
        start_tick = start_bar * ticks_per_bar
        end_tick = start_tick + int(num_bars_in_segment * ticks_per_bar)
        
        new_mid = MidiFile(ticks_per_beat=tpb_target)
        new_track = MidiTrack()
        new_mid.tracks.append(new_track)
        
        # Add tempo and time signature information
        new_track.append(MetaMessage('set_tempo', tempo=tempo, time=0))
        new_track.append(MetaMessage('time_signature', numerator=time_signature[0], denominator=time_signature[1], clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))

        pre_segment_msgs = []
        post_segment_msgs = []
        current_tick = 0
        for msg in target_mid.tracks[0]:
            current_tick += msg.time
            if current_tick < start_tick:   
                pre_segment_msgs.append(msg)
            elif current_tick > end_tick:
                post_segment_msgs.append(msg)

        segment_msgs = []
        for msg in segment_mid.tracks[0]:
            if msg.type == 'note_on':
                segment_msgs.append(msg)

        def ensure_all_notes_off(messages):
            active_notes = set()
            for msg in messages:
                if msg.type == 'note_on':
                    if msg.velocity > 0:
                        active_notes.add(msg.note)
                    else:
                        active_notes.discard(msg.note)
        
            note_off_messages = []
            for note in active_notes:
                note_off_messages.append(mido.Message('note_on', note=note, velocity=0, time=0))
            
            return messages + note_off_messages
        
        pre_segment_msgs = ensure_all_notes_off(pre_segment_msgs)
        segment_msgs = ensure_all_notes_off(segment_msgs)

        new_track.extend(pre_segment_msgs)
        new_track.extend(segment_msgs)
        new_track.extend(post_segment_msgs)
        
        new_mid.save(dest_path)
        os.remove(scaled_segment_path)
    


In [ ]:
import mido
path = r'C:\Users\cunn2\OneDrive\DSML\Project\thesis-repo\plagdet\data\synthetic_dataset\monophonic_midis\melodies'
# Get a list of all MIDI files in the directory
midi_files = [f for f in os.listdir(path) if f.endswith('.mid') or f.endswith('.midi')]

file = os.path.join(path, midi_files[14])

MIDIPlayer(file, 500, styler=cifka_advanced, title='First Monophonic Melody')

extraction

In [ ]:
gen = SyntheticDataGenerator()

dest = 'dest.midi'
gen.extract_bars(file, dest, 1, 2)  

MIDIPlayer(dest, 500, styler=cifka_advanced, title='Extracted Bars')

[2024-09-05 15:20:29] [INFO ] Original file: BPM: 120.0, Time Signature: (4, 4), Ticks per beat: 220


In [ ]:
final = 'final.midi'
gen.insert_segment(dest, file, 8, final)

MIDIPlayer(final, 500, styler=cifka_advanced, title='Inserted Segment')

In [ ]:
MIDIPlayer(file, 500, styler=cifka_advanced)